In [ ]:
import tensorflow as tf
import numpy as np

# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# One-hot encode labels
y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

# ImageNet normalization constants
imagenet_mean = tf.constant([0.485, 0.456, 0.406], dtype=tf.float32)
imagenet_std = tf.constant([0.229, 0.224, 0.225], dtype=tf.float32)

def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    image = (image - imagenet_mean) / imagenet_std
    return image, label

# Dataset pipeline (no augmentation)
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train_cat))
train_ds = train_ds.shuffle(10000) \
                   .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE) \
                   .batch(64) \
                   .prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test_cat))
test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE) \
                 .batch(64) \
                 .prefetch(tf.data.AUTOTUNE)

# Load MobileNetV2 base model (ImageNet pretrained)
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False  # freeze base


# Add classification head with dropout
inputs = tf.keras.Input(shape=(224,224,3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)  # Dropout added here
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)


# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks for learning rate reduction and early stopping
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
]

# Train only the head for 5 epochs
model.fit(train_ds, epochs=5, validation_data=test_ds, callbacks=callbacks)

# Unfreeze base model for fine-tuning
base_model.trainable = True

# Recompile with lower LR for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune for 5 more epochs
model.fit(train_ds, epochs=5, validation_data=test_ds, callbacks=callbacks)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 161s 187ms/step - accuracy: 0.5851 - loss: 1.2089 - val_accuracy: 0.7740 - val_loss: 0.6617 - learning_rate: 0.0010
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 164s 150ms/step - accuracy: 0.7487 - loss: 0.7281 - val_accuracy: 0.7728 - val_loss: 0.6740 - learning_rate: 0.0010
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 144s 153ms/step - accuracy: 0.7620 - loss: 0.6905 - val_accuracy: 0.7813 - val_loss: 0.6329 - learning_rate: 0.0010
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 140s 151ms/step - accuracy: 0.7667 - loss: 0.6782 - val_accuracy: 0.7893 - val_loss: 0.6208 - learning_rate: 0.0010
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 143s 152ms/step - accuracy: 0.7682 - loss: 0.6718 - val_accuracy: 0.7906 - val_loss: 0.6091 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 5.
Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 277s 280ms/step - accurac

In [ ]:
model.save("mobilenetv2_cifar10.keras")


In [ ]:
import os

file_path = "mobilenetv2_cifar10.keras"
size_mb = os.path.getsize(file_path) / (1024 * 1024)
print(f"Model size: {size_mb:.2f} MB")


Model size: 26.40 MB


In [ ]:
from google.colab import files

# Download the saved model file to your local PC
files.download("mobilenetv2_cifar10.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>